# PART1

## 1. ESTUDIO PREVIO DE LOS ARCHIVOS
En este apartado se realizarán comprobaciones para verificar que la información sobre los campos dada en los PDFs es correcta.
Por ejemplo, el archivo 'actividadeseconomicas' puede tener campos repetidos, es decir, que ya se encuentran en el archivo 'locales'. Por lo que sería bastante eficiente eliminar esos campos duplicados, los cuales pueden ser unidos por su clave foranea ID_LOCAL en el futuro.
Esto proporcionaría velocidad de consulta , ahorro de espacio y de computo a futuro.

In [16]:
#------------Dependencies-----------#
import pandas as pd
import numpy as np
import os

In [17]:
#Path to files
path_actividad = "data_files/actividadeconomica202312.csv"
path_licencias = "data_files/licencias202312.csv"
path_locales = "data_files/locales202312.csv"
path_terrazas = "data_files/terrazas202312.csv"


**Dataframe creation 'Actividad Economica'**

In [18]:
df_actividad = pd.read_csv(
    path_actividad,
    sep = ";",
    skip_blank_lines = True,
    on_bad_lines = 'skip',
    engine = 'python',
    encoding = 'utf-8'
)

**Dataframe creation 'Licencias'**

In [19]:
df_licencias = pd.read_csv(path_actividad,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Locales'**

In [20]:
df_locales = pd.read_csv(path_locales,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Terrazas'**

In [21]:
df_terrazas = pd.read_csv(path_terrazas,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

## 1.1 CAMPOS DUPLICADOS

**Locales vs Actividades**

In [22]:
#Query por 'id_local'
resultado = df_locales.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6    20000756
 Name: id_local, dtype: int64,
 6    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64)

In [23]:
#Query por 'id_local'
resultado = df_actividad.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6515    20000756
 Name: id_local, dtype: int64,
 6515    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64)

In [24]:
#Query por 'id_local'
resultado = df_terrazas.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(5471    20000756
 Name: id_local, dtype: int64,
 5471    4472755.5
 Name: coordenada_y_local, dtype: float64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64)

**Como se puede comprobar hay campos que tienen los datos repetidos en ambos archivos por lo que la información descrita en los PDFs es veraz**
Lo que plantearé será dejar el dataframe "locales" como el archivo maestro y luego eliminaré de los demás archivos las dimensiones que posean datos repetidos, así solo quedarán los campos con información limpia y quedarán las futuras colecciones mejor estructuradas.

En el caso en que se quieran realizar consultas de union de tablas el ID **id_local** servirá con clave foranea en las demás tablas.

**Eliminacion de columnas con información duplicadas**

In [25]:
# 1. DataFrame de Locales (fuente principal)
# No hacemos cambios en df_locales

# 2. DataFrame de Actividades
# Mantener solo campos únicos de actividades
columnas_actividades = [
    'id_local',  # clave de unión
    'id_seccion', 'desc_seccion', 'id_division', 'desc_division',
    'id_epigrafe', 'desc_epigrafe'
]
df_actividad = df_actividad[columnas_actividades]

# 3. DataFrame de Licencias
# Mantener solo campos únicos de licencias (pero no aparecen en el índice proporcionado)
# Asumimos que deberían estar estas columnas basadas en los PDFs:
columnas_licencias = [
    'id_local',  # clave de unión
    'ref_licencia', 'id_tipo_licencia', 'desc_tipo_licencia',
    'ide_tipo_situacion_licencia', 'desc_tipo_situacion_licencia',
    'Fecha_Dec_Lic'
]
# Filtramos solo las columnas que existan realmente
columnas_licencias = [col for col in columnas_licencias if col in df_licencias.columns]
df_licencias = df_licencias[columnas_licencias] if columnas_licencias else df_licencias[['id_local']]

# 4. DataFrame de Terrazas
# Mantener solo campos únicos de terrazas
columnas_terrazas = [
    'id_terraza', 'id_local',  # claves
    'Superficie_ES', 'Superficie_RA', 'mesas_aux_es', 'mesas_aux_ra',
    'mesas_es', 'mesas_ra', 'sillas_es', 'sillas_ra', 'hora_ini_LJ_es',
    'hora_fin_LJ_es', 'hora_ini_LJ_ra', 'hora_fin_LJ_ra', 'hora_ini_VS_es',
    'hora_fin_VS_es', 'hora_ini_VS_ra', 'hora_fin_VS_ra',
    'desc_ubicacion_terraza', 'id_periodo_terraza', 'desc_periodo_terraza',
    'id_situacion_terraza', 'desc_situacion_terraza'
]
df_terrazas = df_terrazas[columnas_terrazas]


## 1.2 Limpieza y análisis de los DF

### 1.2.1 DF_LOCALES

In [35]:
df_locales.describe(include='all')

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,id_local_agrupado,rotulo,cod_postal,hora_apertura1,hora_apertura2,hora_cierre1,hora_cierre2,fx_carga,fx_datos_ini,fx_datos_fin
count,1.511620e+05,151162.000000,151162,151162.000000,151162,151162.000000,151162.000000,151162.000000,151162.000000,1.511620e+05,...,90445,151162,151162.000000,8796,812,6311,862,151162,151162,151162
unique,NaN,NaN,21,NaN,131,NaN,NaN,NaN,NaN,NaN,...,3467,80142,NaN,63,37,56,34,112,1,1
top,NaN,NaN,CENTRO,NaN,SAN ANDRES,NaN,NaN,NaN,NaN,NaN,...,01,SIN ACTIVIDAD,NaN,08:00,10:00,02:00,02:00,2023-12-08 07:01:00.937,2023-12-01,2023-12-01
freq,NaN,NaN,13440,NaN,4092,NaN,NaN,NaN,NaN,NaN,...,20570,39418,NaN,1977,435,3063,461,2434,151162,151162
mean,2.655094e+08,9.939661,NaN,997.419001,NaN,3.452925,10011.069158,71.408396,399836.581200,4.049711e+06,...,NaN,NaN,28025.110689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5.016712e+07,5.625185,NaN,562.402277,NaN,1.862671,5626.998295,48.532575,129534.909558,1.311569e+06,...,NaN,NaN,14.277943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+07,1.000000,NaN,101.000000,NaN,1.000000,1001.000000,1.000000,0.000000,0.000000e+00,...,NaN,NaN,28001.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.704840e+08,5.000000,NaN,505.000000,NaN,2.000000,5085.000000,32.000000,439092.600000,4.470752e+06,...,NaN,NaN,28014.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.800378e+08,10.000000,NaN,1004.000000,NaN,3.000000,10137.000000,65.000000,440758.600000,4.473638e+06,...,NaN,NaN,28025.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.850114e+08,15.000000,NaN,1501.000000,NaN,5.000000,15041.000000,104.000000,443691.620000,4.476700e+06,...,NaN,NaN,28037.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#Revisión general del data frame
print(df_locales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151162 entries, 0 to 151161
Data columns (total 48 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_local                   151162 non-null  int64  
 1   id_distrito_local          151162 non-null  int64  
 2   desc_distrito_local        151162 non-null  object 
 3   id_barrio_local            151162 non-null  int64  
 4   desc_barrio_local          151162 non-null  object 
 5   cod_barrio_local           151162 non-null  int64  
 6   id_seccion_censal_local    151162 non-null  int64  
 7   desc_seccion_censal_local  151162 non-null  int64  
 8   coordenada_x_local         151162 non-null  float64
 9   coordenada_y_local         151162 non-null  float64
 10  id_tipo_acceso_local       151162 non-null  int64  
 11  desc_tipo_acceso_local     151162 non-null  object 
 12  id_situacion_local         151162 non-null  int64  
 13  desc_situacion_local       15

**Es recomendable trabajar con Object en vez de Strings así que todas las columnas tienen formatos alineados con su tipo de contenido o de dato**

In [30]:
print(df_locales.isnull().sum())

id_local                          0
id_distrito_local                 0
desc_distrito_local               0
id_barrio_local                   0
desc_barrio_local                 0
cod_barrio_local                  0
id_seccion_censal_local           0
desc_seccion_censal_local         0
coordenada_x_local                0
coordenada_y_local                0
id_tipo_acceso_local              0
desc_tipo_acceso_local            0
id_situacion_local                0
desc_situacion_local              0
id_vial_edificio                  0
clase_vial_edificio               0
desc_vial_edificio                0
id_ndp_edificio                   0
id_clase_ndp_edificio             0
nom_edificio                      0
num_edificio                      0
cal_edificio                      0
secuencial_local_PC               0
id_vial_acceso                    0
clase_vial_acceso                 0
desc_vial_acceso                  0
id_ndp_acceso                     0
id_clase_ndp_acceso         

#  Informe: Justificación de Valores Nulos en el Dataset `locales.csv`

Este informe detalla las razones por las cuales la presencia de valores nulos en determinados campos del archivo `locales.csv` del Censo de Locales y Actividades del Ayuntamiento de Madrid es **esperable, coherente** y **estructuralmente justificada**.

La documentación técnica consultada incluye:
-  *Estructura_DS_FicheroCLA.pdf*
-  *Estructura_DS_FicheroCLA_LICENCIAS_cp.pdf*
-  *Estructura_DS_FicheroCLA_Terrazas_cp.pdf*

---

##  1. Campos de Agrupación

Los campos relacionados con agrupaciones de locales (como centros comerciales, mercados, etc.) **solo aplican** a locales con tipo de acceso **"Agrupado"** (`id_tipo_acceso_local = 0`).

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `coordenada_x_agrupacion`<br>`coordenada_y_agrupacion` | 136.661 | Solo se completan para locales agrupados. Los locales tipo "Puerta de Calle" o "Asociado" no deben tener estas coordenadas. |
| `id_planta_agrupado` | 1.648 | Aplica solo a locales dentro de agrupaciones. |
| `id_local_agrupado` | 60.717 | Solo se utiliza para identificar locales dentro de agrupaciones. |
| `id_agrupacion`, `nombre_agrupacion`, `id_tipo_agrup`, `desc_tipo_agrup` | 0 (sin nulos) | Estos campos están presentes siempre, pero para locales no agrupados su valor será `"SIN AGRUPACION"` o `-1`, lo cual es correcto. |

 **Conclusión**: La mayoría de locales no están agrupados, por lo que la ausencia de valores en estos campos es completamente normal y coherente con la estructura del censo.

---

##  2. Campos de Horarios

Los campos de apertura (`hora_apertura1`, `hora_apertura2`) están **condicionados por el tipo de actividad del local**.

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `hora_apertura1` | 142.366 | Muchos locales (oficinas, almacenes, despachos) no tienen obligación de registrar horarios. |
| `hora_apertura2` | 150.350 | No todos los locales operan en doble jornada o turno partido. |

 **Conclusión**: Estos campos dependen del tipo de uso del local y no son aplicables en todos los casos.

---

##  3. Campo `cal_acceso`

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `cal_acceso` | 0 | Aunque no hay nulos, el contenido puede estar vacío cuando el número de acceso no requiere calificador (letras como A, B, DUP, etc.). Es opcional según la dirección. |

 **Conclusión**: La omisión de este dato no implica un error, sino la ausencia de un calificador aplicable.

---

##  Recomendación General

Para cualquier análisis, se debe tener en cuenta que:
- Los **valores nulos responden a la lógica del tipo de local**, su acceso, su agrupación y si tiene o no usos especiales (como terrazas o licencias).
- **No deben imputarse ni eliminarse automáticamente**, sino gestionarse según el contexto semántico de cada variable.

---


In [32]:
#Comprobación de la calidad de los Nulos.
print(df_locales['id_tipo_acceso_local'].value_counts())

id_tipo_acceso_local
1     134010
0      14501
12      2651
Name: count, dtype: int64


In [33]:
agrupados = df_locales[df_locales['id_tipo_acceso_local'] == 0]
print(agrupados[['coordenada_x_agrupacion', 'id_planta_agrupado']].isnull().sum())

coordenada_x_agrupacion      0
id_planta_agrupado         989
dtype: int64


In [34]:
puerta_calle = df_locales[df_locales['id_tipo_acceso_local'] == 1]
print(puerta_calle[['coordenada_x_agrupacion', 'id_planta_agrupado']].isnull().sum())

coordenada_x_agrupacion    134010
id_planta_agrupado             12
dtype: int64


**Se va a poner un valor meramente orientativo para rellenar los campos que contienen valores Nulos, con el fin de que a futuro el consumidor obtenga los datos depurados y listos para el consumo**

In [38]:
# Coordenadas de agrupación (float)
df_locales['coordenada_x_agrupacion'] = df_locales['coordenada_x_agrupacion'].fillna(0.0)
df_locales['coordenada_y_agrupacion'] = df_locales['coordenada_y_agrupacion'].fillna(0.0)

# Plantas y números de local (int)
df_locales['id_planta_agrupado'] = df_locales['id_planta_agrupado'].fillna(-99)  # Código especial
df_locales['id_local_agrupado'] = df_locales['id_local_agrupado'].fillna(-99)

# Horarios (asumiendo formato "hh:mm:ss")
df_locales['hora_apertura1'] = df_locales['hora_apertura1'].fillna('00:00:00')
df_locales['hora_apertura2'] = df_locales['hora_apertura2'].fillna('00:00:00')

# Relleno de horarios de cierre (asumiendo formato string "HH:MM:SS")
df_locales['hora_cierre1'] = df_locales['hora_cierre1'].fillna('00:00:00')
df_locales['hora_cierre2'] = df_locales['hora_cierre2'].fillna('00:00:00')

In [40]:
#COMPROBACIÓN FINAL DE NULOS
print(df_locales.isnull().sum())

id_local                     0
id_distrito_local            0
desc_distrito_local          0
id_barrio_local              0
desc_barrio_local            0
cod_barrio_local             0
id_seccion_censal_local      0
desc_seccion_censal_local    0
coordenada_x_local           0
coordenada_y_local           0
id_tipo_acceso_local         0
desc_tipo_acceso_local       0
id_situacion_local           0
desc_situacion_local         0
id_vial_edificio             0
clase_vial_edificio          0
desc_vial_edificio           0
id_ndp_edificio              0
id_clase_ndp_edificio        0
nom_edificio                 0
num_edificio                 0
cal_edificio                 0
secuencial_local_PC          0
id_vial_acceso               0
clase_vial_acceso            0
desc_vial_acceso             0
id_ndp_acceso                0
id_clase_ndp_acceso          0
nom_acceso                   0
num_acceso                   0
cal_acceso                   0
coordenada_x_agrupacion      0
coordena